In [ ]:
"""import pandas as pd

file1 = 'combined-jokes/reddit_jokes_output.csv'
file2 = 'combined-jokes/shortjokes.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

combined_df = pd.concat([df1, df2], ignore_index=True)"""

In [ ]:
"""unique_jokes_df = combined_df.drop_duplicates(subset=['Joke'])

unique_jokes_df = unique_jokes_df.reset_index(drop=True)
unique_jokes_df['ID'] = range(1, len(unique_jokes_df) + 1)

columns = ['ID'] + [col for col in unique_jokes_df.columns if col != 'ID']
unique_jokes_df = unique_jokes_df[columns]

output_file = './combined_jokes.csv'
unique_jokes_df.to_csv(output_file, encoding='utf-8', index=False)

print(f"{output_file}")"""

In [ ]:
"""file_path = "./combined_jokes.csv"
df = pd.read_csv(file_path)
df.info()"""

In [ ]:
"""import re
CONTRACTIONS = {
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "aren't": "are not",
    "doesn't": "does not",
    "don't": "do not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "couldn't": "could not",
    "wasn't": "was not",
    "weren't": "were not",
    "n't": " not",
    "i'm": "i am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "we're": "we are",
    "they're": "they are",
    "i've": "i have",
    "you've": "you have",
    "we've": "we have",
    "they've": "they have",
    "i'll": "i will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "we'll": "we will",
    "they'll": "they will",
    "i'd": "i would",
    "you'd": "you would",
    "he'd": "he would",
    "she'd": "she would",
    "we'd": "we would",
    "they'd": "they would",
    "let's": "let us",
    "that's": "that is",
    "what's": "what is",
    "here's": "here is",
    "there's": "there is",
    "who's": "who is",
    "how's": "how is",
}
sensitive_words = [
    "sex", "gender", "womanizer", "feminist", "chauvinist", "misogynist",
    "misandrist", "patriarchy", "matriarchy",
    
    "racism", "racist", "black", "white", "asian", "african", "indian",
    "nigger", "cracker", "slur", "latino", "hispanic", "jew", "antisemitic",
    "arab", "middle eastern", "xenophobia", "immigrant",
    
    "penis","fuck", "vagina", "breast", "boobs", "dick", "cock", "pussy", "butt",
    "anus", "testicle", "scrotum", "nipple", "semen", "sperm", "condom",
    "masturbate", "ejaculate", "rape", "molest", "abuse", "hooker", 
    "prostitute", "whore", "slut", "porn", "pornography", "fetish",
    "orgy", "orgasm", "clitoris", "labia", "erection", "virginity",
    
    "dumb", "idiot", "moron", "retard", "cripple",
    "gay", "lesbian", "homosexual", "bisexual", "transgender", 
    "queer", "homo", "tranny", "dyke", "fag", "pervert", 
    "harass", "molestation", "pedophile", "incest"
]"""

In [ ]:
"""def clean_text(text):
    text = text.replace('"', '')
    text = re.sub(r"&\w+;", "", text)
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"[^\w\s.,!?']", "", text)
    text = re.sub(r"[^a-zA-Z0-9.,!?'\s]", "", text)
    text = re.sub(r"\.{3,}", ".", text)
    text = re.sub(r"\!{2,}", "!", text)
    text = re.sub(r"\?{2,}", "?", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.lower()
    text = expand_contractions(text)
    
    return text
    
def expand_contractions(text):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in CONTRACTIONS.keys()) + r')\b')
    return pattern.sub(lambda x: CONTRACTIONS[x.group()], text)

def contains_sensitive_words(text):
    return any(word in text for word in sensitive_words)

#def preprocess_text(text):
#    tokens = word_tokenize(text)
#    tokens = [lemmatizer.lemmatize(token) for token in tokens]
#    return tokens"""


In [26]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_path = "./work/fine_tuned_gpt2_small_kaggle_final_1221/" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading fine-tuned model and tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Wrap with DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

model.to(device)
model.eval()


print("Model setup complete!")

Loading fine-tuned model and tokenizer...
Model setup complete!


In [27]:
def generate_joke(prompt, model, tokenizer, device, max_length=100, temperature=0.7, top_k=50, top_p=0.9, repetition_penalty=1.2):
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask, 
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            do_sample=True,  
            pad_token_id=tokenizer.pad_token_id,  
            eos_token_id=tokenizer.eos_token_id  
        )

    joke = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return joke


In [28]:
prompts = [
    "Why did the chicken cross the road?",
    "What’s orange and sounds like a parrot?",
    "Why was the math book sad?",
    "Why don’t skeletons fight each other?"
]

for prompt in prompts:
    joke = generate_joke(prompt, model, tokenizer, device)
    print(f"Prompt: {prompt}")
    print(f"Generated Joke: {joke}")
    print("-" * 50)


Prompt: Why did the chicken cross the road?
Generated Joke: Why did the chicken cross the road? it was a lame excuse.
--------------------------------------------------
Prompt: What’s orange and sounds like a parrot?
Generated Joke: What’s orange and sounds like a parrot? an elephant!
--------------------------------------------------
Prompt: Why was the math book sad?
Generated Joke: Why was the math book sad? it had too many problems.
--------------------------------------------------
Prompt: Why don’t skeletons fight each other?
Generated Joke: Why don’t skeletons fight each other? because they have no guts.
--------------------------------------------------
